In [2]:
def sigma_z_rectangle(x,y,z,s0,a,b):
    xi = np.array([-a/2,-a/2,a/2,a/2])
    yi = np.array([-b/2,b/2,b/2,-b/2])
    Ri = np.sqrt( (x+xi)**2 + (y+yi)**2 + z**2 )
    
    term1i = ( 1/((x+xi)**2 + z**2) + 1/((y+yi)**2 + z**2) ) * ( (x+xi)*(y+yi)*z ) / Ri
    term2i = np.arctan( ( (x+xi)*(y+yi) ) / (Ri * z) )
    signi = np.array([(-1)**i for i in range(0,4)])
    
    sigmai = signi*(term1i + term2i)
    return s0/(2 * np.pi) * sigmai.sum()

In [2]:
#identische Funktion, aber funktioniert mit anderen array strukturen
def sigma_z_rectangle2(x,y,z,s0,a,b):
    xi = np.array([-a/2,-a/2,a/2,a/2])
    yi = np.array([-b/2,b/2,b/2,-b/2])
    sigmai = 0.
    for i in range(0,4):
        Ri = np.sqrt( (x+xi[i])**2 + (y+yi[i])**2 + z**2 )    
        term1i = ( 1/((x+xi[i])**2 + z**2) + 1/((y+yi[i])**2 + z**2) ) * ( (x+xi[i])*(y+yi[i])*z ) / Ri
        term2i = np.arctan( ( (x+xi[i])*(y+yi[i]) ) / (Ri * z) )
        signi = (-1)**i    
        sigmai += signi*(term1i + term2i)
    return s0/(2 * np.pi) * sigmai

In [2]:
def plot_isobars_zz():
    a = 1
    b = 0.4
    fig, ax = plt.subplots(ncols=2,figsize=(24,10))
    xs = np.linspace(-1.2*a,1.2*a,200)
    zs = np.linspace(0.001,2*a,100)
    sigmas = np.zeros((len(xs),len(zs)))
    for i,x in enumerate(xs):
        for j,z in enumerate(zs):
            sigmas[i,j] = sigma_z_rectangle(x,0.0,z,1,a,b)
    X, Z = np.meshgrid(xs,zs)
    cp = ax[0].contour(X, -Z, sigmas.T,levels=10)
    ax[0].clabel(cp, inline=True, 
              fontsize=12)
    ax[0].plot([-a/2,a/2],[0,0],lw=10,color='black')
    ax[0].set_xlabel('$x$ / $a$')
    ax[0].set_ylabel('$z$ / $a$')
    ax[0].set_yticks([0,-1,-2])
    ax[0].set_yticklabels([0,1,2])
    ax[0].set(adjustable='box', aspect='equal')
    #
    ys = np.linspace(-1.2*a,1.2*a,200)
    zs = np.linspace(0.001,2*a,100)
    sigmas = np.zeros((len(ys),len(zs)))
    for i,y in enumerate(ys):
        for j,z in enumerate(zs):
            sigmas[i,j] = sigma_z_rectangle(0.0,y,z,1,a,b)
    Y, Z = np.meshgrid(ys,zs)
    cp = ax[1].contour(Y, -Z, sigmas.T,levels=10)
    ax[1].clabel(cp, inline=True, 
              fontsize=12)
    ax[1].plot([-b/2,b/2],[0,0],lw=10,color='black')
    ax[1].set_xlabel('$y$ / $a$')
    ax[1].set_ylabel('$z$ / $a$')
    ax[1].set_yticks([0,-1,-2])
    ax[1].set_yticklabels([0,1,2])
    ax[1].set(adjustable='box', aspect='equal')
    fig.tight_layout();

In [2]:
def i_Rechteck(z,a,b):
    R = np.sqrt(a**2 + b**2 + z**2)
    term1 = b*z/(b**2+z**2) * a*(R**2 + z**2)/((a**2+z**2)*R)
    term2 = np.arctan( b/z * (a*(a**2+b**2) - 2*a*z * (R-z))/((a**2+b**2)*(R-z) - z*(R-z)**2) )
    return (term1+term2)/(2*np.pi)

In [2]:
def i_K_Rechteck(z,a,b):
    return sigma_z_rectangle2(0.37*a,0.37*b,z,1.0,a,b)

In [1]:
from ipywidgets import widgets
from ipywidgets import interact
def interactive_i_rechteck():
    @interact(#c=widgets.BoundedFloatText(value=0.3,min=0, description='$c$ / MPa'),
               z_to_b=widgets.BoundedFloatText(value=1, min=0.01, max=20., description='$z$ / $b$'),
               a_to_b=widgets.BoundedFloatText(value=2, min=1, max=10., description='$a$ / $b$')
    )
    def plot_MC_states(z_to_b,a_to_b):
        plt.rcParams['axes.spines.right'] = True
        fig, ax = plt.subplots(figsize=(10,10))
        sns.set_palette("Paired",n_colors=5)
        ax2 = ax.twinx()
        z1 = np.linspace(0.001,20,100)
        z2 = np.linspace(0.001,2,100)
        for a in [1,2,3,5,10]:
            ax.plot(i_Rechteck(z1,a,1),-z1,label='%.1f' %a)
            ax2.plot(i_Rechteck(z2,a,1),-z2)
        
        i = i_Rechteck(z_to_b,a_to_b,1)
        ax.set_xlim(0,0.25)
        ax.set_ylim(-20,0)
        ax2.set_ylim(-2,0)
        ax.grid(which='both')
        ax.set_yticks([0,-2,-4,-6,-8,-10,-12,-14,-16,-18,-20])
        ax.set_yticklabels([0,2,4,6,8,10,12,14,16,18,20])
        ax2.set_yticks([0,-.2,-.4,-.6,-.8,-1.0,-1.2,-1.4,-1.6,-1.8,-2.0])
        ax2.set_yticklabels([0,.2,.4,.6,.8,1,1.2,1.4,1.6,1.8,2.0])
        ax.set_xticks(np.arange(0, 0.25, 0.01), minor=True)
        ax.set_yticks(np.arange(0, -20, -1), minor=True)
        ax.set_xlabel('$i = \\Delta \\sigma_{zz}/q_0$')
        ax.set_ylabel('$z$ / $b$')
        ax2.set_ylabel('$z$ / $b$')

        if (z_to_b < 2):
            ax2.plot(i_Rechteck(z2,a_to_b,1),-z2,label='$a/b = %.1f$' %a_to_b,color='black',ls='--')
            ax2.plot([0.25,i,i],[-z_to_b,-z_to_b,0],color='black',ls=':')
            ax2.plot(i,-z_to_b,marker='o',color='red')
        else: 
            ax.plot(i_Rechteck(z1,a_to_b,1),-z1,label='%.1f' %a_to_b,color='black',ls='--')
            ax.plot([0,i,i],[-z_to_b,-z_to_b,0],color='black',ls=':')
            ax.plot(i,-z_to_b,marker='o',color='red')
        ax.text(.1,-10,'$\\Delta \\sigma_{zz} = %.6f \ q_0$' %i)
        ax.spines['bottom'].set_position(('data', 0))
        ax.xaxis.set_label_position('top') 
        ax.xaxis.tick_top()
        fig.legend(loc='upper center',ncol=6)
        plt.show();

In [ ]:
# from ipywidgets import widgets
from ipywidgets import interact
def interactive_i_k_rechteck():
    @interact(#c=widgets.BoundedFloatText(value=0.3,min=0, description='$c$ / MPa'),
               z_to_b=widgets.BoundedFloatText(value=1, min=0.01, max=20., description='$z$ / $b$'),
               a_to_b=widgets.BoundedFloatText(value=2, min=1, max=10., description='$a$ / $b$')
    )
    def plot_MC_states(z_to_b,a_to_b):
        plt.rcParams['axes.spines.right'] = True
        fig, ax = plt.subplots(figsize=(10,10))
        sns.set_palette("Paired",n_colors=5)
        ax2 = ax.twinx()
        z1 = np.linspace(0.001,20,100)
        z2 = np.linspace(0.001,2,100)
        for a in [1,2,3,5,10]:
            ax.plot(i_K_Rechteck(z1,a,1),-z1,label='%.1f' %a)
            ax2.plot(i_K_Rechteck(z2,a,1),-z2)
        
        i = i_K_Rechteck(z_to_b,a_to_b,1)
        ax.set_xlim(0,1)
        ax.set_ylim(-20,0)
        ax2.set_ylim(-2,0)
        ax.grid(which='both')
        ax.set_yticks([0,-2,-4,-6,-8,-10,-12,-14,-16,-18,-20])
        ax.set_yticklabels([0,2,4,6,8,10,12,14,16,18,20])
        ax2.set_yticks([0,-.2,-.4,-.6,-.8,-1.0,-1.2,-1.4,-1.6,-1.8,-2.0])
        ax2.set_yticklabels([0,.2,.4,.6,.8,1,1.2,1.4,1.6,1.8,2.0])
        ax.set_xticks(np.arange(0, 1, 0.05), minor=True)
        ax.set_yticks(np.arange(0, -20, -1), minor=True)
        ax.set_xlabel('$i_\\mathrm{K} = \\Delta \\sigma_{zz}/q_0$')
        ax.set_ylabel('$z$ / $b$')
        ax2.set_ylabel('$z$ / $b$')

        if (z_to_b < 2):
            ax2.plot(i_K_Rechteck(z2,a_to_b,1),-z2,label='$a/b = %.1f$' %a_to_b,color='black',ls='--')
            ax2.plot([1,i,i],[-z_to_b,-z_to_b,0],color='black',ls=':')
            ax2.plot(i,-z_to_b,marker='o',color='red')
        else: 
            ax.plot(i_K_Rechteck(z1,a_to_b,1),-z1,label='%.1f' %a_to_b,color='black',ls='--')
            ax.plot([0,i,i],[-z_to_b,-z_to_b,0],color='black',ls=':')
            ax.plot(i,-z_to_b,marker='o',color='red')
        ax.text(.6,-10,'$\\Delta \\sigma_{zz} = %.6f \ q_0$' %i)
        ax.spines['bottom'].set_position(('data', 0))
        ax.xaxis.set_label_position('top') 
        ax.xaxis.tick_top()
        fig.legend(loc='upper center',ncol=6)
        plt.show();

In [ ]:
# from ipywidgets import widgets
from ipywidgets import interact
def interactive_sohlfuge():
    @interact(#c=widgets.BoundedFloatText(value=0.3,min=0, description='$c$ / MPa'),
               e_to_b=widgets.BoundedFloatText(value=0, min=-0.25, max=0.25, step = 0.05, description='$e$ / $b$')
    )
    def plot_MC_states(e_to_b):
        plt.rcParams['axes.spines.right'] = False
        fig, ax = plt.subplots()
        x_to_b = np.linspace(-0.49,0.49,200)
        sig_to_f_to_b = lambda xtb,etb: 2/np.pi * (1+8*etb*xtb)/(np.sqrt(1-4*xtb*xtb))
        ax.plot(x_to_b,sig_to_f_to_b(x_to_b,e_to_b), label = 'starr')
        ax.axhline(1, label='gleichförmig, schlaff')
        #xk = np.sqrt(1-4/np.pi**2)/2
        ax.set_xlabel('$x$ / $b$')
        ax.set_ylabel('$\sigma_0 / q$')
        ax.set_xlim(-0.5,0.5)
        ax.set_ylim(bottom=0)
        ax.legend()
        fig.tight_layout()
        plt.show();

In [ ]:
def plot_char_pt_strip():
    plt.rcParams['axes.spines.right'] = False
    fig, ax = plt.subplots()
    x_to_b = np.linspace(-0.49,0.49,200)
    sig_to_f_to_b = lambda xtb: 2/np.pi * (1)/(np.sqrt(1-4*xtb*xtb))
    ax.plot(x_to_b,sig_to_f_to_b(x_to_b), label = 'zentrisch, starr')
    ax.axhline(1, label='gleichförmig, schlaff')
    xk = np.sqrt(1-4/np.pi**2)/2
    ax.plot(xk,1,marker='o',color='red')
    ax.plot(-xk,1,marker='o',color='red',label='Kennz. P.')
    ax.set_xlabel('$x$ / $b$')
    ax.set_ylabel('$\sigma_0 / q$')
    ax.set_xlim(-0.5,0.5)
    ax.set_ylim(bottom=0)
    ax.legend()
    fig.tight_layout();

In [ ]:
def plot_char_pt_circle():
    plt.rcParams['axes.spines.right'] = False
    fig, ax = plt.subplots()
    r_to_R = np.linspace(0,0.99,100)
    sig_to_q = lambda rtR: 1/(2*np.sqrt(1-rtR*rtR))
    ax.plot(r_to_R, sig_to_q(r_to_R), label = 'zentrisch, starr')
    ax.axhline(1, label='gleichförmig, schlaff')
    rK = np.sqrt(3)/2
    ax.plot(rK,1,marker='o',color='red',label='Kennz. Radius')
    ax.set_xlabel('$r$ / $R$')
    ax.set_ylabel('$\sigma_0 / q$')
    ax.set_xlim(0,1)
    ax.set_ylim(bottom=0)
    ax.legend()
    fig.tight_layout();

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
def plot_char_pt_rect():
    plt.rcParams['axes.spines.right'] = False
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(projection='3d')
    xta = np.linspace(-0.49,0.49,200)
    ytb = np.linspace(-0.49,0.49,200)
    X, Y = np.meshgrid(xta,ytb)
    #
    sig_to_q = lambda x,y: 4/(np.pi**2 * np.sqrt((1-4*x**2)*(1-4*y**2)))
    ones = np.ones((len(xta),len(ytb)))
    surf = ax.plot_surface(X,Y,sig_to_q(X,Y),label='zentrisch, starr',alpha=0.8,cmap=cm.coolwarm)    
    surf2 = ax.plot_surface(X,Y,ones,color='black',alpha = 0.3,label='gleichförmig, schlaff')
    xK = np.sqrt(1-4/np.pi**2)/2
    ax.plot([xK,xK,-xK,-xK],[xK,-xK,-xK,xK],[1,1,1,1],marker='o',ls='',color='red',markersize=8)
    ax.set_xlabel('$x$ / $a$')
    ax.set_ylabel('$y$ / $b$')
    ax.set_zlabel('$\sigma_0 / q$')
    ax.set_xlim(-0.5,0.5)
    ax.set_ylim(-0.5,0.5)
    ax.set_zlim(bottom=0)
    ax.view_init(elev=20., azim=-35)
    fig.colorbar(surf,shrink=0.5)
    fig.tight_layout();

In [1]:
def interactive_sohlfuge_lin():
    @interact(s0=widgets.IntSlider(value=100, min=50, max=150, description='$\\sigma_{0,0}$ / kPa'),
        s1=widgets.IntSlider(value=100, min=50, max=150, description='$\\sigma_{0,1}$ / kPa'),
        s2=widgets.IntSlider(value=100, min=50, max=150, description='$\\sigma_{0,2}$ / kPa')
    )
    def plot_lin_rect(s0,s1,s2):
        plt.rcParams['axes.spines.right'] = False
        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot(projection='3d')
        xta = np.linspace(-0.5,0.5,100)
        ytb = np.linspace(-0.5,0.5,100)
        X, Y = np.meshgrid(xta,ytb)
        #
        c0 = (s0+s2)/2
        c1 = s2-s1
        c2 = s1-s0
        s3 = s0 - s1 + s2

        sig = lambda x,y: c0 + c1*x + c2*y
        surf = ax.plot_surface(X,Y,sig(X,Y),alpha=0.8,cmap=cm.coolwarm)
        ax.plot(0,0,c0,marker='o',label='$\\bar{\\sigma}_0$',ls='')
        ax.plot(-0.5,-0.5,s0,marker='o',label='$\\sigma_{0,0}$',ls='')
        ax.plot(-0.5,0.5,s1,marker='o',label='$\\sigma_{0,1}$',ls='')
        ax.plot(0.5,0.5,s2,marker='o',label='$\\sigma_{0,2}$',ls='')
        ax.plot(0.5,-0.5,s3,marker='o',label='$\\sigma_{0,3}$',ls='')
        ax.set_xlabel('$x$ / $a$')
        ax.set_ylabel('$y$ / $b$')
        ax.set_zlabel('$\sigma_0$ / kPa')
        ax.set_xlim(-0.5,0.5)
        ax.set_ylim(-0.5,0.5)
        ax.set_zlim(bottom=0)
        #ax.view_init(elev=20., azim=-35)
        fig.legend()
        fig.colorbar(surf,shrink=0.5)
        fig.tight_layout()
        plt.show();

In [2]:
def sig_scaling(c_vec,*args):
    c0 = c_vec[0]
    c1 = c_vec[1]
    c2 = c_vec[2]
    x = args[0]
    y = args[1]
    V_to_ab = args[2]
    exi = args[3]
    eeta = args[4]
    sig = lambda y,x: np.maximum(c0 + c1*x + c2*y,0) #y must be first argument
    sigx = lambda y,x: np.maximum(c0 + c1*x + c2*y,0)*x #y must be first argument
    sigy = lambda y,x: np.maximum(c0 + c1*x + c2*y,0)*y #y must be first argument
    r1 = scp.integrate.dblquad(sig, -0.5, 0.5, lambda x: -0.5, lambda x: 0.5)[0] - V_to_ab #force equilibrium
    r2 = scp.integrate.dblquad(sigx, -0.5, 0.5, lambda x: -0.5, lambda x: 0.5)[0] - V_to_ab*exi #moment equilibrium y
    r3 = scp.integrate.dblquad(sigy, -0.5, 0.5, lambda x: -0.5, lambda x: 0.5)[0] - V_to_ab*eeta #moment equilibrium y
    return [r1,r2,r3]

In [2]:
def sig_scaled(x,y,c0,c1,c2,V_to_ab,exi,eeta):
    arg = (x,y,V_to_ab,exi,eeta)
    res = scp.optimize.root(fun=sig_scaling,x0=[0,0,0],args=arg)
    c0 = res.x[0]
    c1 = res.x[1]
    c2 = res.x[2]
    sig = lambda y,x: np.maximum(c0 + c1*x + c2*y,0)
    return sig(y,x)

In [2]:
def interactive_sohlfuge_lin_V(kw=False,kw2=False,scaled=False):
    @interact(V_to_ab=widgets.IntSlider(value=100, min=0, max=200, step=5, description='$V$ / $ab$'),
        exi=widgets.FloatSlider(value=0, min=-0.4, max=0.4, step=0.05, description='$e_x$ / $a$'),
        eeta=widgets.FloatSlider(value=0, min=-0.4, max=0.4, step=0.05, description='$e_y$ / $b$')
    )
    def plot_lin_rect_V(V_to_ab,exi,eeta):
        plt.rcParams['axes.spines.right'] = False
        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot(projection='3d')
        xta = np.linspace(-0.5,0.5,100)
        ytb = np.linspace(-0.5,0.5,100)
        X, Y = np.meshgrid(xta,ytb)
        #
        s0 = V_to_ab * (1 - 6*exi - 6*eeta)
        s1 = V_to_ab * (1 - 6*exi + 6*eeta)
        s2 = V_to_ab * (1 + 6*exi + 6*eeta)
        s3 = s0 - s1 + s2
        #   
        c0 = (s0+s2)/2
        c1 = s2-s1
        c2 = s1-s0
        #
        sig = lambda x,y: c0 + c1*x + c2*y
        if (scaled):
            surf = ax.plot_surface(X,Y,sig_scaled(X,Y,c0,c1,c2,V_to_ab,exi,eeta),alpha=0.8,cmap=cm.coolwarm)
            #for cut stress distributions
        else:
            surf = ax.plot_surface(X,Y,sig(X,Y),alpha=0.8,cmap=cm.coolwarm)
        ax.plot(exi,eeta,sig(exi,eeta),marker='x',color='black',ls='',markersize=10)
        ax.plot([exi,exi],[eeta,eeta],[sig(exi,eeta)+V_to_ab,sig(exi,eeta)],color='black')
        ax.plot([exi,exi,0.5],[eeta,eeta,eeta],[sig(exi,eeta),0,0],color='black',ls=':')
        ax.plot([exi,exi],[eeta,-0.5],[0,0],color='black',ls=':')
        #ax.quiver(exi,eeta,sig(exi,eeta),0,0,-V_to_ab,length=10,normalize=True)
        ax.plot([-0.5,-0.5,0.5,0.5,-0.5],[-0.5,0.5,0.5,-0.5,-0.5],[0,0,0,0,0],color='black',label='Fundamentbegrenzung')
        if (kw):
            x = np.linspace(-1/6,1/6,100)
            y = lambda x: 1/6 - np.abs(x)
            ax.plot(x,y(x),0,color='red',label='1. Kernweite')
            ax.plot(x,-y(x),0,color='red')
        if (kw2):
            x = np.linspace(-1/3,1/3,100)
            y = lambda x: np.sqrt(1/9 - x**2)
            ax.plot(x,y(x),0,color='green',label='2. Kernweite')
            ax.plot(x,-y(x),0,color='green')
        ax.set_xlabel('$x$ / $a$')
        ax.set_ylabel('$y$ / $b$')
        ax.set_zlabel('$\sigma_0$ / kPa')
        ax.set_xlim(-0.5,0.5)
        ax.set_ylim(-0.5,0.5)
        ax.set_zlim(bottom=0)
        #ax.view_init(elev=20., azim=-35)
        fig.legend()
        fig.colorbar(surf,shrink=0.5)
        fig.tight_layout()
        plt.show();

In [ ]:
def sig_kern_1D(e,x):
    if (np.abs(e) <= 1/6):
        return 1+12*e*x
    else:
        se = 2/(3*(1/2 - np.abs(e)))
        x0 = np.sign(e)*(3*np.abs(e) -1)
        return np.maximum(np.sign(e)*se/(np.sign(e)*(-np.sign(x0)*np.abs(x0)) +0.5)*(x-x0),0)

In [ ]:
def interactive_kern2():
    @interact(exi=widgets.FloatSlider(value=0, min=-0.45, max=0.45, step=0.01, description='$e_x$ / $a$')
    )
    def plot_kern2(exi):
        fig, ax = plt.subplots(figsize=(14,8))
        xi = np.linspace(-0.5,0.5,100)
        ax.plot(xi,sig_kern_1D(0,xi),label='$e_x = 0$',ls='--',alpha=0.5)
        ax.plot(xi,sig_kern_1D(-1/6,xi),label='$e_x = -a/6$',ls='--',alpha=0.5)
        ax.plot(xi,sig_kern_1D(1/6,xi),label='$e_x = a/6$',ls='--',alpha=0.5)
        ax.plot(xi,sig_kern_1D(-1/3,xi),label='$e_x = -a/3$',ls='--',alpha=0.5)
        ax.plot(xi,sig_kern_1D(1/3,xi),label='$e_x = a/3$',ls='--',alpha=0.5)
        ax.plot(xi,sig_kern_1D(exi,xi),label='$e_x = %.1f a$' %exi,color='black')
        ax.axvline(-1/6,ls=':',color='grey',alpha=0.8,label='1. Kernweite')
        ax.axvline(1/6,ls=':',color='grey',alpha=0.8)
        ax.axvline(-1/3,ls='-.',color='grey',alpha=0.8,label='2. Kernweite')
        ax.axvline(1/3,ls='-.',color='grey',alpha=0.8)
        ax.plot([exi,exi],[0,0.5],color='black')
        ax.set_xlabel('$x$ / $a$')
        ax.set_ylabel('$\\sigma_0$ / $V\,(ab)^{-1}$')
        ax.set_xlim(-0.5,0.5)
        ax.set_ylim(bottom=0)
        fig.legend()
        fig.tight_layout()
        plt.show();

In [ ]:
def plot_lin_rect_V(V_to_ab,exi,eeta):
    plt.rcParams['axes.spines.right'] = False
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(projection='3d')
    xta = np.linspace(-0.5,0.5,200)
    ytb = np.linspace(-0.5,0.5,200)
    X, Y = np.meshgrid(xta,ytb)
    #
    s0 = V_to_ab * (1 - 6*exi - 6*eeta)
    s1 = V_to_ab * (1 - 6*exi + 6*eeta)
    s2 = V_to_ab * (1 + 6*exi + 6*eeta)
    s3 = s0 - s1 + s2
    #   
    c0 = (s0+s2)/2
    c1 = s2-s1
    c2 = s1-s0
    #
    sig = lambda x,y: c0 + c1*x + c2*y
    surf = ax.plot_surface(X,Y,sig_scaled(X,Y,c0,c1,c2,V_to_ab,exi,eeta),alpha=0.8,cmap=cm.coolwarm)

    ax.plot(exi,eeta,sig(exi,eeta),marker='x',color='black',ls='',markersize=10)
    ax.plot([exi,exi],[eeta,eeta],[sig(exi,eeta)+V_to_ab,sig(exi,eeta)],color='black')
    ax.plot([exi,exi,0.5],[eeta,eeta,eeta],[sig(exi,eeta),0,0],color='black',ls=':')
    ax.plot([exi,exi],[eeta,-0.5],[0,0],color='black',ls=':')
    #ax.quiver(exi,eeta,sig(exi,eeta),0,0,-V_to_ab,length=10,normalize=True)
    ax.plot([-0.5,-0.5,0.5,0.5,-0.5],[-0.5,0.5,0.5,-0.5,-0.5],[0,0,0,0,0],color='black',label='Fundamentbegrenzung')
    x = np.linspace(-1/6,1/6,100)
    y = lambda x: 1/6 - np.abs(x)
    ax.plot(x,y(x),0,color='red',label='1. Kernweite')
    ax.plot(x,-y(x),0,color='red')
    x = np.linspace(-1/3,1/3,100)
    y = lambda x: np.sqrt(1/9 - x**2)
    ax.plot(x,y(x),0,color='green',label='2. Kernweite')
    ax.plot(x,-y(x),0,color='green')
    ax.set_xlabel('$x/a$')
    ax.set_ylabel('$y/b$')
    ax.set_zlabel('$\sigma_0$ / kPa')
    ax.set_xlim(-0.5,0.5)
    ax.set_ylim(-0.5,0.5)
    ax.set_zlim(bottom=0)
    #ax.view_init(elev=20., azim=-35)
    fig.legend()
    fig.colorbar(surf,shrink=0.5)
    fig.tight_layout()
    plt.show();

In [ ]:
style = {'description_width': 'initial'}
def interactive_kern_compound():
    @interact(exi = widgets.FloatSlider(value=0.2, min=-0.45, max=0.45, step=0.01, description='$e_x$ / $a$'),
              excav = widgets.FloatSlider(value=30, min=0, max=150, step=1, description='$\\gamma_d d$ / $V\,(ab)^{-1}$',style=style)
    )
    def plot_kern2(exi,excav):
        fig, ax = plt.subplots(figsize=(20,8),ncols=2)
        xi = np.linspace(-0.5,0.5,100)
        #Sohlspannung
        ax[0].plot(xi,sig_kern_1D(exi,xi),label='$\\sigma_{zz0}$',color='black')
        #Mittlere Sohlspannung
        ax[0].plot(xi,sig_kern_1D(0,xi),label='$\\bar{\\sigma}_{zz0} = V\,(ab)^{-1}$',ls='--')
        #Vorbelastung
        ax[0].plot(xi,sig_kern_1D(0,xi)*(excav/100),label='$\\gamma_d d$',ls='--')
        #Erstbelastung
        xi_pos = xi[np.where(sig_kern_1D(exi,xi)-sig_kern_1D(0,xi)*(excav/100)>0)]
        xi_neg = xi[np.where(sig_kern_1D(exi,xi)-sig_kern_1D(0,xi)*(excav/100)<=0)]
        ax[0].fill_between(xi_pos,sig_kern_1D(exi,xi_pos),sig_kern_1D(0,xi_pos)*(excav/100),color='blue',alpha=0.2,label='Erstbelastung')
        ax[0].fill_between(xi_neg,sig_kern_1D(exi,xi_neg),sig_kern_1D(0,xi_neg)*(excav/100),color='red',alpha=0.2,label='Entlastung')
        #
        ax[0].axvline(-1/6,ls=':',color='grey',alpha=0.8,label='1. Kernweite')
        ax[0].axvline(1/6,ls=':',color='grey',alpha=0.8)
        ax[0].axvline(-1/3,ls='-.',color='grey',alpha=0.8,label='2. Kernweite')
        ax[0].axvline(1/3,ls='-.',color='grey',alpha=0.8)
        #
        mean_s1 = sig_kern_1D(0,xi)*(1-excav/100)
        s1 = sig_kern_1D(exi,xi) - sig_kern_1D(0,xi)*excav/100
        delta_mean_s1 = s1 - mean_s1
        rechteck = mean_s1-np.max(delta_mean_s1)
        max_dreieck = np.max(mean_s1+np.max(delta_mean_s1))
        min_dreieck = np.min(mean_s1-np.max(delta_mean_s1))
        
        ax[1].plot(xi,s1,label='$\\sigma_{zz1} = \\sigma_{zz0} - \\gamma_d d$')
        if (exi >= 0 and exi <= 1/6):
            if (rechteck[0]*max_dreieck >= 0):
                ax[1].fill_between(xi,rechteck,0,label='Rechteckanteil',color='blue',alpha=0.2)
                ax[1].fill_between([-0.5,0.5],[rechteck[0],max_dreieck],rechteck[0:1],label='Dreiecksanteil',color='red',alpha=0.2)
            else:
                ax[1].fill_between([-0.5,0.5],[0,max_dreieck],0,label='Dreiecksanteil',color='red',alpha=0.2)
                ax[1].fill_between([-0.5,0.5],[min_dreieck,0],0,label='Dreiecksanteil',color='blue',alpha=0.2)
        elif (exi <0 and exi >= -1/6):
            if (rechteck[0]*max_dreieck >= 0):
                ax[1].fill_between(xi,rechteck,0,label='Rechteckanteil $+$',color='blue',alpha=0.2)
                ax[1].fill_between([-0.5,0.5],[max_dreieck,rechteck[0]],rechteck[0:1],label='Dreiecksanteil',color='red',alpha=0.2)
            else:
                ax[1].fill_between([-0.5,0.5],[max_dreieck,0],0,label='Dreiecksanteil',color='red',alpha=0.2)
                ax[1].fill_between([-0.5,0.5],[0,min_dreieck],0,label='Dreiecksanteil',color='blue',alpha=0.2)
        elif (exi > 1/6):
            x0 = np.sign(exi)*(3*np.abs(exi) -1)
            ax[1].fill_between(xi,np.min(s1),0,label='Rechteckanteil',color='blue',alpha=0.2)
            ax[1].fill_between([x0,0.5],[0,max_dreieck],0,label='Dreiecksanteil',color='red',alpha=0.2)
        elif (exi < 1/6):
            x0 = np.sign(exi)*(3*np.abs(exi) -1)
            ax[1].fill_between(xi,np.min(s1),0,label='Rechteckanteil',color='blue',alpha=0.2)
            ax[1].fill_between([-0.5,x0],[max_dreieck,0],0,label='Dreiecksanteil',color='red',alpha=0.2)
            
        for i in range(2):
            ax[i].plot([exi,exi],[0,0.5],color='black')
            ax[i].set_xlabel('$x$ / $a$')
            ax[i].set_ylabel('$\\sigma$ / $V\,(ab)^{-1}$')
            ax[i].set_xlim(-0.5,0.5)
            ax[i].legend()
        
        ax[0].set_ylim(bottom=0)
        fig.tight_layout()
        plt.show();